# Text summarization: TextRank

With the enormous amount of data surrounding us, it is important to be able to extract the most important information from it. In this notebook, we focus on one such information extraction algorithm from text. 

Broadly speaking, there are two different approaches for summarizing text: 1) extractive summarization, where the summary is lifted verbatim from the document itself and 2) abstractive summarization, where the summary is not a part of the document but is synthesized de novo by a learning model. 

Abstractive summarization is an extremely difficult problem and remains an area of active research. Some of the recent advancements employ recurrant neural networks and can be found in the works mentioned in [Quora](https://www.quora.com/Has-Deep-Learning-been-applied-to-automatic-text-summarization-successfully).

Here, I approach text summarization from an extractive viewpoint, tackling two specific questions: 
   * Given a document, which are the most important sentences?   
   * Given a document, what are the key words?  

To answer these questions, I implement TextRank - a graph-based algorithm that ranks text in a document based on the importance of the text. TextRank is analogous to Google's PageRank and was introduced by Mihalcea and Tarau in the paper [TextRank: Bringing Order into Texts](https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf).

TextRank is an unsupervised learning algorithm and much simpler to implement as compared to abstractive summarization and yet yields good Recall-Oriented Understudy for Gisting Evaluation (ROUGE) scores. 

## Utility

In [257]:
import logging
import re
from IPython.core.display import display, HTML
import numpy as np
import argparse
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [258]:
# Utility functions

def get_sentences(doc):
    sentence_tokenizer = PunktSentenceTokenizer()
    return sentence_tokenizer.tokenize(doc)

def remove_non_words(sentences):
    regex = re.compile('[^a-zA-Z\" "]')
    return [regex.sub('', s) for s in sentences]

def get_idx_to_sentences(sentences):
    return {idx: s for idx, s in enumerate(sentences)}

def get_idx_to_word(vocab):
    return {vocab[word]: word for word in vocab}

def get_ranks(directed_graph_weights, d=0.85):
    A = directed_graph_weights
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0
        col_sum = np.sum(A[:,id])
        if col_sum != 0:
            A[:, id] /= col_sum
        A[:, id] *= -d
        A[id, id] = 1
    
    B = (1-d) * np.ones((matrix_size, 1))
    
    ranks = np.linalg.solve(A, B)
    return {idx: r[0] for idx, r in enumerate(ranks)}

def display_highlighted_sentneces(ranks_of_sentences, 
                                  raw_sentneces, 
                                  sentences_to_highlight = 3,
                                  dark=0.8):
    sorted_sentences_ranks_idx = sorted(ranks_of_sentences, key=lambda k: ranks_of_sentences[k], reverse=True)
    weights = [ranks_of_sentences[idx] for idx in ranks_of_sentences]
    weights = (weights - min(weights))/(max(weights) - min(weights) + 1e-4)
    html = ''
    fmt = ' <span style="background-color: #{0:x}{0:x}ff">{1}</span>'
    for idx in range(len(raw_sentences)):
        if idx in sorted_sentences_ranks_idx[:sentences_to_highlight]:
            c = int(256*((1.-dark)*(1.-ranks_of_sentences[idx])+dark))
        else:
            c = int(256*((1.-dark)*(1.-0)+dark))    
        html += fmt.format(c,raw_sentences[idx])
    display(HTML(html))
    
def display_highlighted_words(ranks_of_words, 
                              raw_sentences, 
                              vocab,
                              words_to_highlight = 10,
                              dark=0.8):
    weights = [ranks_of_words[idx] for idx in ranks_of_words]
    sorted_words_ranks_idx = sorted(ranks_of_words, key=lambda k: ranks_of_words[k], reverse=True)
    weights = (weights - min(weights))/(max(weights) - min(weights) + 1e-4)
    html = ''
    fmt = ' <span style="background-color: #{0:x}{0:x}ff">{1}</span>'
    for s in raw_sentences:
        for w_ in s.split(' '):
            regex = re.compile('[^a-zA-Z\" "]')
            w = regex.sub('', w_)
            if len(PorterTokenizer().__call__(w))!=0:
                stemmed_word = PorterTokenizer().__call__(w)[0].lower()
            else:
                stemmed_word = " "
            if stemmed_word in vocab and vocab[stemmed_word] in sorted_words_ranks_idx[:words_to_highlight]:
                c = int(256*((1.-dark)*(1.-ranks_of_words[vocab[stemmed_word]])+dark))
            else:
                c = int(256*((1.-dark)*(1.-0)+dark))
            html += fmt.format(c,w_)
    display(HTML(html))

In [259]:
logger = logging.getLogger('TextRank')
logger.setLevel(logging.DEBUG)

In [260]:
simple_doc = "Quantum mechanics is interesting. Quantum mechanics is weird. Hello, you there?"
document = simple_doc

simple_doc commentry: For illustrative purpose, we will use the above simple_doc to show the steps involved in the implementation of TextRank. The following things should be noted about this document:
   * It is clear that the third sentence is not something important. So, we expect that the third sentence should be ranked lowest by TextRank. 
   * It is not clear whether the first or the second sentence is more important. 
   * It is clear that "quantum" and "mechanics" are the most important words.

In [261]:
raw_sentences = get_sentences(document) # From the document, extract the list of sentences
sentences = remove_non_words(raw_sentences) # Remove all non-words from raw_sentences
idx_to_sentences = get_idx_to_sentences(sentences) # Get index to sentences 

logger.debug(sentences)

DEBUG:TextRank:['Quantum mechanics is interesting', 'Quantum mechanics is weird', 'Hello you there']


In [262]:
# A callable class which stems the word to its root according to the rules defined in ProterStemmer
class PorterTokenizer(object):
    def __init__(self):
        self.porter = PorterStemmer()

    def __call__(self, *args, **kwargs):
        return [self.porter.stem(word) for word in args[0].split()]
    
logger.debug(PorterTokenizer().__call__("run running runs")) # Example

DEBUG:TextRank:[u'run', u'run', u'run']


## TFIDF

In [263]:
# We create a term frequency-inverse-document-frequency vectorizer object
# Input: List of sentences.
# Processing: 1) Remove stop words defined in stop_words from the sentences and 
#             2) Stem the words to its roots according to PorterStemmer
tfidf = TfidfVectorizer(preprocessor=None, 
                        stop_words=stopwords.words('english'),
                        tokenizer=PorterTokenizer())

In [264]:
# tfidf_mat: Normalized tfidf matrix with each row corresponding to a sentence and each column corresponding to a word 
# vocab: Dictionary of words and its corresponding index. 
#        The index coresponds to the column number of the word in tfidf_mat 
tfidf_mat = tfidf.fit_transform(sentences).toarray()

logger.debug('\n{}'.format(tfidf_mat))

DEBUG:TextRank:
[[ 0.    0.68  0.52  0.52  0.  ]
 [ 0.    0.    0.52  0.52  0.68]
 [ 1.    0.    0.    0.    0.  ]]


simple_doc commentry: We see from the above that there are 5 words which make up our vocabulary and there are three sentences. Notice that the words "you" and "there", which were part of the third sentence: "Hello you there?", have been removed from the vocabulary by stop_words. As a result of this, only "hello" remains in the third sentence. This is confirmed by the fact that for the third sentence (third row), we have 1 in the 0th column (note that in vocab, 'hello': 0) of tfidf_mat and all other columns are zero.

In [265]:
cv = CountVectorizer(preprocessor=None, 
                        stop_words=stopwords.words('english'),
                        tokenizer=PorterTokenizer())
cv_mat = normalize(cv.fit_transform(sentences).toarray().astype(float), axis=0)
vocab = cv.vocabulary_
idx_to_word = get_idx_to_word(vocab)

logger.debug('\n{}'.format(idx_to_word))

DEBUG:TextRank:
{0: u'hello', 1: u'interest', 2: u'mechan', 3: u'quantum', 4: u'weird'}


## Construct directed weighed graph

For the purpose of carrying out the algorithm of TextRank, we now construct a directed weighed graph where each sentence is a node and the edges between two sentences specify the similarity between them. Suppose s_i corresponds to tfidf vector for sentence i (that is the i_th row in tfidf_mat), then the similarity between sentence i and j is defined as s_i * s_j.T

In [266]:
# directed_graph_weights_sentences: A square matrix with dimension (num_of_sentences x num_of_sentences).
#                                 : This matrix is symmetric. 
#                                 : (i,j)th element of the matrix specifies the similarity between sentences i and j. 
directed_graph_weights_sentences = np.dot(tfidf_mat, tfidf_mat.T)

logger.debug('\n{}'.format(directed_graph_weights_sentences))

DEBUG:TextRank:
[[ 1.    0.54  0.  ]
 [ 0.54  1.    0.  ]
 [ 0.    0.    1.  ]]


Similar to defining the weight graph for sentences, we can define a weight graph for the words in the document. The similarity between words i and j is defined as s_i.T * s_j where s_i and s_j are sentence rows in tfidf_mat. 

In [267]:
# directed_graph_weights_words: A square matrix with dimension (num_of_words_in_vocab x num_of_words_in_vocab).
#                             : This matrix is symmetric. 
#                             : (i,j)th element of the matrix specifies the similarity between words i and j. 
directed_graph_weights_words = np.dot(cv_mat.T, cv_mat)

logger.debug('\n{}'.format(directed_graph_weights_words))

DEBUG:TextRank:
[[ 1.    0.    0.    0.    0.  ]
 [ 0.    1.    0.71  0.71  0.  ]
 [ 0.    0.71  1.    1.    0.71]
 [ 0.    0.71  1.    1.    0.71]
 [ 0.    0.    0.71  0.71  1.  ]]


## Ranks of sentences/words using PageRank 

Now that we have the graph weights, we solve for the ranks of the sentences and words in the document. 

In [268]:
ranks_of_sentences = get_ranks(directed_graph_weights_sentences, 0.85)
ranks_of_words = get_ranks(directed_graph_weights_words, 0.85)

logger.debug(ranks_of_sentences)
logger.debug(ranks_of_words)

DEBUG:TextRank:{0: 1.0, 1: 1.0, 2: 0.15000000000000002}
DEBUG:TextRank:{0: 0.15000000000000002, 1: 0.76495280978071989, 2: 1.2350471902192799, 3: 1.2350471902192799, 4: 0.76495280978071978}


## Hilighted text

In [269]:
display_highlighted_sentneces(ranks_of_sentences, raw_sentences)

In [270]:
display_highlighted_words(ranks_of_words, raw_sentences, vocab)

## Summary

   * In this notebook we implemented TextRank using tfidf matrix as the basis for similarity measure.  
   * This extractive summarization method can be the starting point for abstractive summarization. For example, suppose we have a document having around 100 sentences. Making an abstractive model (say by using recurrent neural network) which can take these 100 sentences as input, and "generate" a headline from these 100 sentences is extremely challenging because of the large input space. This inout space can be dramatically reduced by first using TextRank to extract, say 5, most important sentneces from the document and then using these 5 sentences as input to the abstractive summarizer model.  
   * In this work, I did not test how well the model performs. I tried using pyrouge, but got into configuration (ini) issues. 
   * For key-word ranking, it was noted in the original TextRank paper that it is better to use only nouns and adjectives as possible candidates. In this work, besides using sentence tokanization and removing stop words, I did not use any grammar specific knowledge. It will be interesting to implement grammar based filtering.